In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
!wget $data

--2025-10-12 16:43:13--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-12 16:43:13 (7.72 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [4]:
df = pd.read_csv('course_lead_scoring.csv')

In [5]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [109]:
df.shape

(1462, 9)

## Initial Data preparation and cleaning
*Missing Values*
* Categorical - NaN --> 'NA'
* Numerical - NaN --> 0

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [75]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count','lead_score']

In [11]:
for c in categorical:
    df[c] = df[c].fillna('NA')

for n in numerical:
    df[n] = df[n].fillna(0)

In [12]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Q1
*What is the most frequent observation for the **industry** column?*  **retail is the most frequent**

In [18]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Q2 Create a Correlation matrix for numerical features.  Which two have the largest correlation?
**Number of courses viewed/annual income with interaction count have the largest correlation**

In [25]:
df[['interaction_count','number_of_courses_viewed']].corrwith(df.lead_score)

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [26]:
df[['number_of_courses_viewed','annual_income']].corrwith(df.interaction_count)

number_of_courses_viewed   -0.023565
annual_income               0.027036
dtype: float64

In [27]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)  #this only splits into 2 pieces.  to get train/val, must split again

In [28]:
#splitting full into train/val.  Note that to get 20% of full data, you need 25% of the previously split df_full_train
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42) 

In [29]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98


In [110]:
df_train.shape

(876, 8)

In [38]:
df_full_train = df_full_train.reset_index(drop = True)

## Q3.  Calculate mutual information score
*Which variable has the biggest score?* **lead_source has the biggest score**

In [43]:
from sklearn.metrics import mutual_info_score

In [44]:
def mutual_info_converted_score(series):
    return mutual_info_score(df_full_train.converted, series)

In [45]:
df_full_train[categorical].apply(mutual_info_converted_score).sort_values(ascending = False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

## Q4 Train a Logistic Regression
*What is the accuracy of the prediction?* **Incorrect answer since it is on the train vs. validation set, but based on feedback it is 0.74**

In [46]:
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')

In [48]:
dv = DictVectorizer(sparse = False)

In [91]:
X_train = dv.fit_transform(train_dicts)

In [92]:
val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')

In [93]:
X_val = dv.transform(val_dicts)

In [52]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [53]:
model.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [54]:
y_pred = model.predict_proba(X_val)[:,1]

In [63]:
converted_decision = (y_pred > 0.5)
(y_val == converted_decision).mean().round(4)

np.float64(0.6997)

In [65]:
y_pred2 = model.predict_proba(X_train)[:,1]

In [85]:
conv_dec = (y_pred2 > 0.5)
og_accuracy = (y_train == conv_dec).mean().round(2)
og_accuracy

np.float64(0.74)

## Q5 Find least useful feature
**lead_score has the smallest difference**

In [67]:
feats = ['industry', 'employment_status', 'lead_score']

In [106]:
#for f in feats:
for f in list(categorical + numerical):
    full = list(categorical + numerical)
    full.remove(f)
    filtered_train_dicts = df_train[full].to_dict(orient = 'records')
    filteredX_train = dv.fit_transform(filtered_train_dicts)
    filtered_val_dicts = df_val[full].to_dict(orient = 'records')
    filteredX_val = dv.transform(filtered_val_dicts)
    model.fit(filteredX_train, y_train)
    y_pred = model.predict_proba(filteredX_val)[:,1]
    converted_decision = (y_pred > 0.5)
    print ('Removed ' + f +' ',(og_accuracy - (y_val == converted_decision).mean()))

Removed lead_source  0.03692832764505116
Removed industry  0.04034129692832766
Removed employment_status  0.043754266211604054
Removed location  0.03010238907849827
Removed number_of_courses_viewed  0.1836860068259386
Removed annual_income  -0.11324232081911267
Removed interaction_count  0.1836860068259386
Removed lead_score  0.03351535836177477


## Q6 Training a regularized logistic regression

In [112]:
for c in [0.01, 0.1, 1, 10, 100]:
    reg_model = LogisticRegression(penalty = 'l2', solver='liblinear', C=c, max_iter=1000, random_state=42)
    reg_model.fit(X_train, y_train)
    y_pred = reg_model.predict_proba(X_val)[:,1]
    converted_decision = (y_pred > 0.5)
    print(c, (y_val == converted_decision).mean().round(4))


0.01 0.6997
0.1 0.6997
1 0.6997
10 0.6997
100 0.6997
